신경망 만들기

==37==

Dezero에서 구현한 함수들은 입력과 출력이 모두 스칼라라고 가정했음

텐서처리) x가 텐서일 경우 - sin함수가 원소별로 적용됨.
입력과 출력텐서의 형상은 바뀌지않는다.

역전파를 할때 텐서를 이용해도 문제없이 동작을 했음.
텐서가 입력으로 들어가서 sum이란 함수로 하나의 스칼라값이 된다.
스칼라 값을 역전파하여 다시 정상적으로 동작을 하나 살펴봐야함.

마지막 출력이 스칼라인 계산 그래프에 대한 역전파
*y.backward(retain_grad=True)를 실행하면 각 변수의 미분값이 구해진다.
*기울기의 형상과 순전파 때의 데이터의 형상이 일치

In [13]:
import numpy as np
import dezero.functions as F
from dezero import Variable
from dezero.core import Function

In [4]:
x =  Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


In [5]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


In [6]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
y = x+c #sum을 취함
print(y) #하나의 스칼라값

variable([[11 22 33]
          [44 55 66]])


In [7]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t =  x + c
y  =F.sum(t)

In [8]:
y.backward(retain_grad=True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


==38==

문제가 발생. 
텐서를 사용한 계산에서의 역전파> 텐서를 입력해도 역전파가 올바르게 성립을 함.
원소별로 계산하지않는 함수
1. reshape함수(텐서의 형상을 변환)
2. transpose함수(행렬을 전치하는)
3. 두함수 모두 텐서의 형상을 바꾸는 함수

In [ ]:
텐서의 형상을 바꾸는 함수인 reshape함수를 사용한다.
텐서의 원소 수는 같고 형상만 변환한다.
-reshape 역전파를 구현
기울기의 형상이 입력의 형상과 같아지도록 변환.
기울기를 x,data.shape와 x.grad.shape가 일치하도록 변환.

reshape함수를 사용하여 형상을 변환시키고 y.backward(retain_grad=True)를 수행하여 x의 기울기를 구함
y의 기울기도 자동으로 채워짐 > 이 형상은 y와 같다.
원소는 모두 1로 이루어진 텐서.

입력값을 기억해놨다가 역전파일때 입력값을 reshape을 해주어서 역전파가 가능한것.

In [9]:
x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x,(6,))
print(y)

[1 2 3 4 5 6]


In [14]:
class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape #입력변수 쪽의 형상에 맞게 변환
        y = x.reshape(self.shape)
        return y

    def backward(self, gy): #grad값을 넣어주면 (2,3)텐서로 바뀐다. (역전파의 기능)
        return reshape(gy, self.x_shape) 

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

In [15]:
def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

def reshape(self, *shape):
    if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
        shape = shape[0]
    return dezero.functions.reshape(self, shape) 과 비슷하게 만드는것.
    
Varible클래스에 가변 인수를 받는 reshape메서드를 추가하고 
reshape함수를 Variable인스턴스의 메서드 형태로 호출을 한다.

In [16]:
x =  np.random.rand(1,2,3)

y = x.reshape((2,3)) #튜플로 받기
y = x.reshape([2,3]) #리스트로 받기
y = x.reshape(2,3) #인수를 그대로(풀어서) 받기

>>reshape 함수 총정리

Dezero용 reshape을 보면 class forward, backward를 구현하고 forward에 입력한 값을 기억하여서 reshape하여 입력과 똑같이 만들어준다.
(2,3)->(6,)로 변환
넘파이의 reshape은 튜플,리스트, 인수를 그대로 풀어서 받는다.
Dezero의 reshape도 같은걸 받기 위해서 class Reshape을 Variable클래스에 넣는다.
def reshape이란 함수가 저장된 dezero.functions

행렬을 전치해주는 함수를 구현
(2,3)^T를 하면 (3,2)로 전치가 된다.

In [17]:
x = np.array([[1,2,3],[4,5,6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


dezero에서도 전치가 가능하게끔 구현해보자.

*numpy의 전치
텐서의 원소 자체는 그대로이고 형상만 바뀐다.
역전파에서는 출력 쪽에서 전해지는 기울기의 형상만 변경한다.
순전파 때와는 정확히 반대형태!

backward(self,gy)가 있다면 return 2x*gy를 예전에 했었다.
하지만 이와 같은 전치경우에는 형상만 바뀐다!! 
출력의 gy를 그대로 받고 입력시에 받은 reshape을 그대로 받고 있다가 다시 resahpe을 해서 형상이 복원이 되었다.


In [18]:
class Transpose(Function):
    def forward(self,x):
        y = np.transpose(x)
        return y
    def backward(self,gy):
        gx = transpose(gy)
        return gx
def transpose(x):
    return Transpose()(x)

In [19]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


In [24]:
x =  Variable(np.random.rand(1,2,3))
y = x.reshape((2,3))
y = x.reshape(2,3)

축!
np.transpose함수의 범용적 사용
A, B, C, D = 1, 2,3,4
x = np.random.rand(A,B,C,D)
y = x.transpose(1,0,3,2)

축의 순서를 지정> 데이터의 축이 달라진다.
인수를 None으로 주면 축이 역순으로 정렬

==39==

Dezero에 합계를 구하는 함수 sum추가
-덧셈의 미분
-sum 함수의 미분을 이끌어낸다.
-sum 함수 구현

덧셈의 미분 > y = x0+x1일때 y'(x0으로 미분)=1 , y'(x1로 미분)=1
역전파는 출력 쪽에서 전해지는 기울기를 그대로 입력 쪽으로 흘려보낸다.

덧셈을 수행한 후 변수y로 부터 역전파한다.

*원소가 2개로 구성된 벡터 합의 역전파
벡터에 sum함수를 적용하면 스칼라를 출력

*원소가 2개 이상인 벡터의 합에 대한 역전파
기울기 벡터의 원소 수만큼 복사하면 되는데 기울기를 입력 변수의 형상과 같아지도록 복사
입력 변수가 2차원 이상의 배열일 때도 동일하게 적용

*Dezero의 Sum클래스와 sum함수 구현
sum함수 역전파에서는 입력 변수의 형상과 같아지게 기울기의 원소를 복사
입력형상을 갖고있는걸 브로드캐스트했다가 복원을 했다.

class Sum(Function):
     def forward(self, x):
        self.x_shape = x.shape
        y = x.sum()
        return y

     def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx
def sum(x):
    return Sum()(x)

sum을 하여 21이 되었는데 복원을 해야하는 과정에서 broadcast에 x.shape을 넣었더니 기울기 gy의 원소를 복사

In [28]:
x = Variable(np.array([1,2,3,4,5,6]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([1 1 1 1 1 1])


In [29]:
x =  Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([[1 1 1]
          [1 1 1]])


*axis(축) 지정 인수
합계를 구할때 축을 지정할 수 있음
x의 형상은 (2,3)이고 출력 y의 형상은(3, )이다.
Axis는 축을 뜻하며, 다차원 배열에서 화살표의 방향을 의미

In [30]:
x = np.array([[1,2,3],[4,5,6,]])
y = np.sum(x, axis=0)
print(y)
print(x.shape, '->', y.shape)

[5 7 9]
(2, 3) -> (3,)


*keepdims인수
:입력과 출력의 차원수(축 수)를 똑같게 유지
    keepdims = True 축의 수가 유지
    keepdims = False y의 형상은 스칼라

In [31]:
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, keepdims=True)
print(y)
print(y.shape)

[[21]]
(1, 1)


==40==

*넘파이 브로드캐스트
서로다른 형상을 가진 배열들간에 산술 연산을 수행하기 위해 배열의 형상을 조정
> 작은 배열을 큰 배열에 맞춰 연산을 수행할 수 있음

*Dezero에서도 브로드캐스트 지원
sum함수를 구현시 역전파에서 구현되지 않은 broadcast_to함수를 이용했음

In [32]:
x = np.array([1,2,3])
y = np.broadcast_to(x,(2,3))#(3,)형상을 (2,3)형상으로 바꿔주었다.
print(y) 

[[1 2 3]
 [1 2 3]]


브로드캐스트가 수행된 후의 역전파는
'원소복사'가 일어날 경우 역전파때는 입력x의 형상과 같아지도록 기울기의 '합'을 구한다.
> sum_to라는 함수로 순전파와 역전파의 관계가 만들어진다.

In [33]:
from dezero.utils import sum_to

x = np.array([[1,2,3],[4,5,6]])
y = sum_to(x,(1,3))
print(y)

y = sum_to(x,(2,1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


*sum_to함수의 역전파
역전파는 broadcast_to함수를 그대로 이용
이 함수를 사용하여 입력 x의 형상과 같아지도록 기울기의 원소를 복제한다.

*BroadcastTo클래스와 broadcast_to함수 구현
역전파에서는 입력 x와 형상을 일치시키는데 Dezero의 sum_to함수 이용

*SumTo클래스와 sum_to함수 구현
역전파에서는 입력 x와 형상이 같아지도록 기울기의 원소를 복제함.
상호의존적 관계인 broadcast_to함수와 sum_to함수

브로드캐스트란?
형상이 다른 다차원 배열끼리 연산을 가능하게 하는 넘파이 기능

sum_to함수 구현 이유: 넘파이 브로드캐스트에 대응하기 위해
    x0,x1은 형상이 다르지만 계산 과정에서 x1의 원소가 x0형상에 맞춰 복제된다.
    
Dezero에서의 브로드캐스트)
순전파는 브로드캐스트 발생(but, 역전파는 일어나지 않는다.)

넘파이의 브로드캐스트)
broadcast_to 함수에서 이뤄지면서 역전파는 sum_to함수에서 일어난다.

*브로드캐스트 역전파 계산
순전파때 브로드캐스트가 일어난다면 x0과 x1의 형상이 다르다는것
>> 두 형상이 다를때 브로드캐스트용 역전파를 계산

기울기gx0는 x0의 형상이 되도록 합을 구하고 기울기 gx1은 x1의 형상이 되도록 합을 구한다.

In [34]:
x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1.grad)

variable([11 12 13])
variable([3])


==41==

In [ ]:
*벡터의 내적
a = (a0,---,an)
b = (b0,---,bn)
내적은 ab = a0b0+---+anbn
두 벡터 사이의 대응 원소의 곱을 모두 합한 값이 벡터의 내적

*행렬의 곱
왼쪽 행렬의 가로방향 벡터와 오른쪽 행렬의 세로방향 벡터 사이의 내적을 계산
벡터의 내적과 행렬의 곱 계산은 모두 np.dot함수로 처리할 수 있음.

In [36]:
#벡터의 내적
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.dot(a,b)
print(c)

a=np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
c = np.dot(a,b)
print(c)

32
[[19 22]
 [43 50]]


In [ ]:
*행렬과 벡터 사용시 체크사항
형상에 주의!
축의 원소의 수 일치

In [ ]:
*행렬 곱의 역전파
최종적으로 스칼라를 출력하는 계산을 다룬다.
L의 각 변수에 대한 미분을 역전파로 구한다.
xi에서 L에 이르는 연쇄 법칙의 경로는 여러개가 있다.
y의 각 원소의 변화를 통해 궁극적으로 L이 변화하게 된다.

라운드x와 라운드y를 알고싶다 > 라운드x는 라운드y와 W^T를 곱해주고 라운드W는 x^T와 라운드y를 곱해주면된다.

In [ ]:
*행렬의 곱 코드 구현